In [11]:
from PIL import Image
from torchvision import transforms
import torch
from PIL import Image
import numpy as np
import torch.nn as nn
from collections import OrderedDict
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms


RESCALE_SIZE = 224
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
PATH_MODEL = r'C:\Users\user\PycharmProjects\flask_test\flask_service\model_resnet34_7ep.pt'
PATH_IMAGE = r'C:\Users\user\PycharmProjects\flask_test\test_files\n02086240_10604.JPEG'

NEU_MODEL = []
DICT_BREEDS = {
    0: 'Australian terrier',
    1: 'Beagle',
    2: 'Border terrier',
    3: 'Dingo',
    4: 'English foxhound',
    5: 'Golden retriever',
    6: 'Old English sheepdog',
    7: 'Rhodesian ridgeback',
    8: 'Samoyed',
    9: 'Shih-Tzu'
}

# Download model
def __download_model(path):
    if NEU_MODEL == []:
        NEU_MODEL.append(models.resnet34(pretrained=True))
        NEU_MODEL[0].fc = nn.Sequential(OrderedDict([
            ('batch_norm', nn.BatchNorm1d(512)),
            ('drop1', nn.Dropout(p=0.5)),
            ('linear1', nn.Linear(512, 10))
            ]))
        NEU_MODEL[0].load_state_dict(torch.load(path))
        NEU_MODEL[0].eval()
        NEU_MODEL[0].to(DEVICE)

# Prepare image from input path
def __prepare_data(path=None):
    image = Image.open(path).convert('RGB')
    tfms = transforms.Compose([
        transforms.Resize(RESCALE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    img_tensor = tfms(image).to('cuda').unsqueeze(0)
    return img_tensor

# Count score
def __count_score(img_path):
    __download_model(path=PATH_MODEL)
    image = __prepare_data(path=img_path)
    output = NEU_MODEL[0](image).cpu()
    
    # If the difference between the values is less than the threshold, then this indicates that the image does 
    # not contain the desired dog breeds. (In this simple way, I'm trying to reduce the likelihood of neural 
    # network cheating). The threshold is needed due to the use of the softmax-function.
    np_arr = output.detach().numpy()
    if np.max(np_arr) - np.min(np_arr) <= 300.:
        return {'score': 'Very noisy image. No dog find!'}
    
    answer = nn.functional.softmax(output.detach(), dim=-1).numpy()
    index = np.argmax(answer)
    return {'breed': DICT_BREEDS[index]}

    
NOISE = r'C:\Users\user\PycharmProjects\flask_test\test_service\test_files\noise.jpg'   
PATH_IMAGE = r'C:\Users\user\PycharmProjects\flask_test\test_service\test_files\n02086240_10604.JPEG'
__count_score(img_path=NOISE)

{'breed': 'Dingo'}

In [7]:
__count_score(img_path=PATH_IMAGE)

{'breed': 'Old English sheepdog'}

In [8]:
n = r'C:\Users\user\PycharmProjects\flask_test\test_files\ILSVRC2012_val_00020379.JPEG'
__count_score(img_path=n)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\PycharmProjects\\flask_test\\test_files\\ILSVRC2012_val_00020379.JPEG'

In [ ]:
m = r'C:\Users\user\PycharmProjects\flask_test\test_files\rasteniya-lekarstva-dikie-derevya-01.jpg'
__count_score(img_path=m)

In [12]:
q = r'C:\Users\user\PycharmProjects\flask_test\test_service\test_files\Buffy_dog.jpeg'
__count_score(img_path=q)

{'breed': 'Old English sheepdog'}